In [1]:
import numpy as np
from pyquil.quil import Program, Pragma
from pyquil.api import get_qc
from pyquil.gates import *
import numpy as np

In [6]:
p = Program()
ro = p.declare('ro', 'BIT', 1)
p += H(0)
p += Pragma('DELAY', [0], str(200e-9))
p += MEASURE(0, ro[0])
print(p)
qc = get_qc('4q-qvm', execution_timeout=1000)
p = qc.compiler.quil_to_native_quil(p)
print(p)
qc.qam.gate_noise=(0.2, 0.0, 0.0)
qc.run(p)


DECLARE ro BIT[1]
H 0
PRAGMA DELAY 0 "2e-07"
MEASURE 0 ro[0]

DECLARE ro BIT[1]
RZ(pi) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
MEASURE 0 ro[0]
HALT



QAMExecutionResult(executable=<pyquil.quil.Program object at 0x000001CAF379F8D0>, readout_data={'ro': array([[0]])}, execution_duration_microseconds=None)